# Generate main figures

This notebook generates the main figures of the article "The Structure and Statistics of Language jointly shape Cross-frequency Dynamics during Spoken Language Comprehension", H. Weissbart & AE. Martin.

The following code rely on the Source Data accompnaying the article. The Source Data is available from the the figshare repository: https://doi.org/10.6084/m9.figshare.16668207

## Content

1. [Figure 2](#Figure-2)
2. [Figure 3](#Figure-3)
3. [Figure 4](#Figure-4)
4. [Figure 5](#Figure-5)

__Note__

> Figure 1 is a schematic representation of the analysis pipeline and is not generated by the present notebook, however we supply another notebook to generate most of the panels of that figure.
> Figure 6 of the article is generated with simulations and is not included in the present notebook, however we supply another notebook to generate the simulations and the accompanying figure.

## Libraries

Libraries and parameters used in the notebook. We will also add the code from `audiobook` to the Python path to be able to use the functions from the package.

We assume that all required libraries are installed (this include the `pyeeg` package).

In [11]:
import sys
sys.path.append('../audiobook')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from audiobook.utils import extract_story_parts_data, STORIES, DATA_PATH, subjects, STIM_PATH
from audiobook.features import get_acoustic_envelope, get_wordlevel_aligned
from matplotlib import font_manager
import pandas as pd 

# Figure parameters
plt.style.use('default')
plt.rcParams['figure.figsize'] = [7.2, 5.4]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.family'] = 'sans'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['font.size'] = 7
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['savefig.transparent'] = True
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.dpi'] = 300


## Colors
featsets = [['wordonsets'],
            ['wordonsets', 'depth', 'close'],
            ['wordonsets', 'surprisal', 'entropy'],
            ['wordonsets', 'surprisal', 'entropy', 'depth', 'close']]

colors = {'wordonsets': (0.3,)*3}
for k in featsets:
    key = '_'.join(k)
    is_predict = any([f in key for f in ['surprisal', 'wordfrequency']])
    is_syntax = any([f in key for f in ['open', 'depth', 'close']])
    if is_predict and not is_syntax:
        if 'entropy' in key:
            colors[key] = sns.blend_palette(['purple', 'seagreen', 'orange'], n_colors=5)[2]
        else:
            colors[key] = sns.blend_palette(['purple', 'seagreen', 'orange'], n_colors=5)[3]
    if is_syntax and not is_predict:
        colors[key] = sns.blend_palette(['purple', 'seagreen', 'orange'], n_colors=5)[0]
    if is_syntax and is_predict:
        colors[key] = sns.blend_palette(['purple', 'seagreen', 'orange'], n_colors=5)[-1]


colors = {'wordonsets': (0.3,)*3}
for k in featsets:
    key = '_'.join(k)
    is_predict = any([f in key for f in ['surprisal', 'wordfrequency']])
    is_syntax = any([f in key for f in ['open', 'depth', 'close']])
    if is_predict and not is_syntax:
        if 'entropy' in key:
            colors[key] = sns.blend_palette(['#7A4B9B', '#2AAD9D', 'orange'], n_colors=5)[2]
        else:
            colors[key] = sns.blend_palette(['#7A4B9B', '#2AAD9D', 'orange'], n_colors=5)[3]
    if is_syntax and not is_predict:
        colors[key] = sns.blend_palette(['#7A4B9B', '#2AAD9D', 'orange'], n_colors=5)[0]
    if is_syntax and is_predict:
        colors[key] = sns.blend_palette(['#7A4B9B', '#2AAD9D', 'orange'], n_colors=5)[-1]

paper_fontsizes = {
    'axes.labelsize': 7,
    'axes.titlesize': 7,
    'xtick.labelsize': 6,
    'ytick.labelsize': 6,
    'legend.fontsize': 7
}

C:\Users\hugwei\Desktop\feature-PAC\audiobook\text.py:170: SyntaxWarning: invalid escape sequence '\w'
  wordlvl = wordlvl[wordlvl.token.str.contains('\w')].reset_index() # new line of code: does it break the code for WF??!


## Figure 2

This figures presents the summary of stimulus statistics and MEG spectra:

- Power spectra of MEG
- power spectra of audio fr vs nl
- coherence
- ITPC and Power modulation

In [12]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
norm = mcolors.LogNorm(vmin=0.01, vmax=10)
norm = mcolors.Normalize(vmin=0.01, vmax=10)
from scipy.stats import ttest_1samp

from pyeeg.vizu import topomap
from tqdm.notebook import tqdm
import glob
import os
from functools import reduce
import mne
from mne.stats import fdr_correction
from audiobook.utils import SUBJECTS

paper_fontsizes = {
    'axes.labelsize': 7,
    'axes.titlesize': 7,
    'xtick.labelsize': 6,
    'ytick.labelsize': 6,
    'legend.fontsize': 7
}

from scipy.signal import welch
from scipy.stats import ttest_ind
from scipy.stats.distributions import t as tdist

c:\users\hugwei\documents\pyeeg\pyeeg\simulate.py:246: SyntaxWarning: invalid escape sequence '\ '
  """


In [13]:
# Load relevant data
# Load power spectra
data = np.load('all_psds.npz', allow_pickle=True)
Pxx = {}
for k in data.keys():
    Pxx[k] = data[k].item()
data.close()

# Load coherence data
nl_cohs = []
fr_cohs = []
for f in glob.glob('./Data/coherence_nl_fr_sub*.npz'):
    data = np.load(f)
    nl_cohs.append(data['nl_coh'])
    fr_cohs.append(data['fr_coh'])
    data.close()
# Need to fix the number of channels?
nl_coh = np.mean(nl_cohs, 0)
fr_coh = np.mean(fr_cohs, 0)

# Fooof data
df = pd.read_pickle('Data/Fooof/sensor_global_fooof_results.pkl')
results = df.to_dict()

# Parameters
info_269 = mne.io.read_info(os.path.join(DATA_PATH, 'processed', 'sub-026', 'meg', 'audioBook-filtered-ICAed-raw.fif'))
info_269 = mne.pick_info(info_269, mne.pick_types(info_269, meg=True, ref_meg=False))
# Info with 267 channels:
bads = ['MLT41', 'MRO52']
info = mne.io.read_info(os.path.join(DATA_PATH, 'processed', 'sub-026', 'meg', 'audioBook-filtered-ICAed-raw.fif'))
info = mne.pick_info(info, mne.pick_types(info, meg=True, ref_meg=False, exclude=bads))

fs = 200
nfreqbins = 1024
freqs = np.fft.rfftfreq(nfreqbins, 1/fs)

FileNotFoundError: [Errno 2] No such file or directory: 'all_psds.npz'